# 2.3 Integration with MongoDB databases

Instead of storing the index in a variable or in a plain-text file, we could export it to any tool of our choice.
For convenience, **signac** provides export routines for MongoDB database collections.

**Please note: The following steps require pymongo and either a local MongoDB instance or a signac database configuration.**

In [1]:
import signac

#
# --- PLEASE UNCOMMENT ONE OF THE FOLLOWING TWO CONFIGURATIONS ---
#

# --- 1) USING A LOCAL MONGODB INSTANCE ---
#from pymongo import MongoClient
#client = MongoClient()
#db = client.testing

# --- 2) USING A SIGNAC DATABASE CONFIGURATION ---
db = signac.get_database('testing')

index = db.signac_tutorial_index
index.drop()
master_index = db.signac_tutorial_master_index
master_index.drop()

We can export either the project index directly via:

In [2]:
project = signac.get_project(root='projects/tutorial')

signac.contrib.export_pymongo(project.index(), index)

Or we compile and export a master index:

In [3]:
master_crawler = signac.contrib.MasterCrawler(root='projects')
signac.contrib.export_pymongo(master_crawler.crawl(), master_index)

In both cases we can now use MongoDB's query engine for advanced search queries.
For example, let's find the mean gas volume of water for the pressure range (2.0 < p <= 5.0):

In [4]:
import numpy as np

query = {
    '$and': [{'statepoint.p': {'$gt': 2.0}}, {'statepoint.p': {'$lte': 5.0}}],
    'fluid': 'water',
    'V_gas': {'$exists': True}}
docs = master_index.find(query)
V = np.array([doc['V_gas'] for doc in docs])
print(V.mean())

158.435561567
